In [2]:
# ---------------------------------------------------
# 2. Decorator that keeps local state
# ---------------------------------------------------
def call_counter(func):
    """Decorator that counts calls to *func*."""
    count = 0
    def wrapper(*args, **kwargs):
        nonlocal count
        count += 1
        print(f"[{func.__name__}] call #{count}")
        return func(*args, **kwargs)
    return wrapper

@call_counter
def greet(name):
    print("Hello,", name)

greet("Alice")
greet("Bob")
@call_counter
def say_hi(name):
    print("Hi there,", name)

say_hi("Mallory")
say_hi("Eve")

[greet] call #1
Hello, Alice
[greet] call #2
Hello, Bob
[say_hi] call #1
Hi there, Mallory
[say_hi] call #2
Hi there, Eve


Absolutely! This is a **beautiful example** of both:

- **Decorators**
- **Closures**

We’ll walk through this code **step-by-step**, just like we did with the `power_factory`, so you understand how everything connects and why closures are essential here.

---

## 🧠 What Is a Decorator?

A **decorator in Python** is simply a function that:
- Takes another function as input
- Returns a new (or modified) function

It’s used to **wrap behavior around an existing function**, without modifying its source code.

Think of it like wrapping a gift:
- The original function is the gift
- The decorator is the wrapper, adding extra functionality (like logging or timing)

---

## 🔍 Let's Look at the Code Step by Step

### 🟦 1. The Decorator Function: `call_counter(func)`

```python
def call_counter(func):
    """Decorator that counts calls to *func*."""
    count = 0
```

This is the **outer function** — our decorator.

- It takes one argument: `func` → the function being decorated.
- Inside it, we define a variable `count = 0` → this will be our counter.
- So far, nothing else has happened. Just setting things up.

---

### 🟨 2. The Wrapper Function

```python
    def wrapper(*args, **kwargs):
        nonlocal count
        count += 1
        print(f"[{func.__name__}] call #{count}")
        return func(*args, **kwargs)
```

Now we define the **inner function**: `wrapper`.

This function:
- Accepts any arguments (`*args`, `**kwargs`) so it can wrap any function.
- Uses `nonlocal count` → this tells Python: “I want to modify the `count` variable from the outer scope.”
- Increments the counter each time it runs.
- Prints out which function is being called and how many times.
- Then calls the original function (`func`) with the same arguments.

> 💡 This `wrapper` function is what replaces the original function (`greet`) after decoration.

---

### 🟩 3. Return the Wrapper Function

```python
    return wrapper
```

The decorator returns the `wrapper` function.

So now, when someone calls the decorated function (like `greet()`), they're actually calling this `wrapper`.

---

## 🎉 Now the Decorator Is Used

```python
@call_counter
def greet(name):
    print("Hello,", name)
```

This line:

```python
@greet_counter
```

is equivalent to:

```python
greet = call_counter(greet)
```

Which means:
- We pass the `greet` function into `call_counter`
- `call_counter` returns the `wrapper` function
- And we assign that back to `greet`

So now:
- Every time you call `greet(...)`, you're really calling the `wrapper(...)` function
- Which increments the counter, prints a message, and then runs the real `greet`

---

## 🧱 Step-by-Step Execution

Let’s run the full example:

```python
greet("Alice")
greet("Bob")
```

### First Call: `greet("Alice")`

1. You call `greet("Alice")`
2. But `greet` now refers to the `wrapper` function
3. Inside `wrapper`:
   - `count` was 0 before
   - `count += 1` → now `count = 1`
   - Print: `[greet] call #1`
   - Then call the original `greet("Alice")` → "Hello, Alice"
4. Done!

### Second Call: `greet("Bob")`

1. Again, `greet("Bob")` goes to `wrapper`
2. Inside `wrapper`:
   - `count` was 1
   - `count += 1` → now `count = 2`
   - Print: `[greet] call #2`
   - Then call original `greet("Bob")` → "Hello, Bob"

And so on…

---

## 🧠 Why Is This a Closure?

Because:
- The inner `wrapper` function uses the `count` variable defined in the **enclosing scope** (`call_counter`)
- Even though `call_counter` finishes running after decoration, `wrapper` still remembers and modifies `count`

That’s exactly what a **closure** does:
- A function that remembers variables from the scope in which it was defined

Here:
- `count` lives in `call_counter`
- `wrapper` closes over `count`
- That makes `wrapper` a closure

---

## 🔍 Inspecting the Closure (Optional but Cool!)

You can inspect the closure like this:

```python
print(greet.__closure__)
# Output: (<cell at 0x...: int object at 0x...>, <cell at 0x...: function object at 0x...>)
```

Each item in `__closure__` corresponds to a captured variable.

To see the current value of `count`:

```python
print(greet.__closure__[0].cell_contents)  # Should show 2 if called twice
```

---

## ✅ Summary Table

| Part | What It Does |
|------|---------------|
| `call_counter(func)` | Outer decorator function; sets up state (`count`) |
| `count = 0` | State variable that persists across calls |
| `wrapper(*args, **kwargs)` | Inner function that wraps the original function |
| `nonlocal count` | Allows modification of the outer `count` variable |
| `return wrapper` | Returns the wrapped function |
| `@call_counter` | Syntactic sugar for reassigning `greet = call_counter(greet)` |
| Closure? | ✅ Yes! `wrapper` remembers and modifies `count` from `call_counter` |

---

## 🧪 Try It Yourself

You can try decorating multiple functions:

```python
@call_counter
def say_hi(name):
    print("Hi there,", name)

say_hi("Mallory")
say_hi("Eve")
```

Output:
```
[say_hi] call #1
Hi there, Mallory
[say_hi] call #2
Hi there, Eve
```

Each decorated function gets its own independent counter — thanks to closures!

---

## 🧩 Final Thought

This pattern — using a decorator with a closure to keep track of state — is very powerful.

You’re seeing:
- How decorators let you add behavior to functions
- How closures let you keep private state between calls
- All without global variables or classes

